<a href="https://colab.research.google.com/github/SetHet/TranscriptorCharlas/blob/main/TranscriptorCharlas_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transcriptor de Charlas para multiples agentes.

Este proyecto se basa en este Google Colab [transcripts with speaker names](https://colab.research.google.com/drive/1V-Bt5Hm2kjaDb4P1RyMSswsDKyrzc2-3?usp=sharing#scrollTo=O0_tup8RAyBy). Realize mejoras para mejorar la precision y generar archivos de texto. Dividi el archivo entrante de audio y lo procese en fragmentos para corregir un error de transcripcion que afectaba a audios demaciado extensos.

Este proyecto esta diseñado para ser ejecutado en Google Colab con sus ambientes de GPU.

Notas de uso:

- Utilizar el modo GPU de Google Colab, Recomendado T4 GPU. Esto se encuentra en "Entorno de ejecución/Cambiar tipo de entorno de ejecucion/T4 GPU" (https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm).
- La transcripcion se guardara en formato TXT y HTML y se descargaran automaticamente, puede que el explorador pida permiso de multi descargas.
- Cambie el numero de hablantes segun el audio entregado.
- Seleccione un modelo más grande si tu quieres más precision y un modelo menor si tu quieres que se ejecute rapidamente. Para T4_GPU se puede usar el modelo de mayor tamaño ([más información](https://github.com/openai/whisper#available-models-and-languages)).
- Si el lenguaje es ingles, selecciona el idioma, sino el general funciona bastante bien, especialmente en español.


Vista de alto nivel de lo que ocurre en el codigo:


1.   Se sube un audio por el sistema de google colab, este proceso puede tardar bastante segun el tamaño del archivo.
2.   Se instalaran los paquetes necesarios para el funcionamiento y cargara el modelo en memoria de video.
3.   Se separara el audio en trozos que se procesaran individualmente para mejorar la precision de la transcripcion. Esto se realiza con PyDub.
4.   Se procesaran lada trozo en varios segmentos de audio y generaran las transcripciones. Esto se realiza con Whisper de OpenAI.
5.   Se reagruparan los trozos de audio y texto, para su procesamiento de deteccion de hablantes. Esto se realiza mediante CLusters de aglomeración.
6.   Se generara un archivo de texto y otro de html para una mejor lectura.

Espero que te sea util. Dime si esto puede ser mejorado.


Notes on usage:

- Make sure to [change runtime to GPU](https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm).
- The transcript will be saved in Files, which you can find in the menu on the left.
- Change the number of speakers below if different from two.
- Pick a bigger model if you want more accuracy and a smaller model if you want the program to run faster ([more info](https://github.com/openai/whisper#available-models-and-languages)).
- If you know the language being spoken is English, then change language to 'English' as this improves performance.


High level overview of what's happening here:


1.   I'm using Open AI's Whisper model to seperate audio into segments and generate transcripts.
2.   I'm then generating speaker embeddings for each segments.
3.   Then I'm using agglomerative clustering on the embeddings to identify the speaker for each segment.

Let me know if I can make it better!


# Configuracion

In [ ]:
path = "audio.wav"
version_tscript = 10 #@param {type: "integer"}

num_speakers = 2 #@param {type:"integer"}

language = 'any' #@param ['any', 'English', 'es']

model_size = 'large-v2' #@param ['tiny', 'base', 'small', 'medium', 'large', 'large-v2']

clips_time_min = 10 #@param {type: "integer"}
direction_clips = 'clips/' #@param {type:"string"}

direction_text = 'texts/' #@param {type:"string"}

model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


# Cargar Audio
Al ejecutarse esta sección hay que cargar el audio

In [ ]:
# upload audio file

from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

# Instalacion de bibliotecas

In [ ]:
!pip uninstall torch torchvision torchaudio torchdata torchtext triton -y
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 torchtext==0.15.2  triton==2.0.0 torchdata==0.6.1
!pip install openai cohere tiktoken
!pip install speechbrain==0.5.16
print("p1")
#!pip install -q git+https://github.com/openai/whisper.git@v20230918 > /dev/null
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
print("p2")
#!pip install -q git+https://github.com/pyannote/pyannote-audio@b7960890aefda70e9595609d52e61e7e61fb16d7 > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
print("p3")
!pip install PyDub
!pip install lightning-fabric

# Importaciones

In [ ]:

nombre_archivo = path

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

# Cargar modelo Whisper

In [ ]:
model = whisper.load_model(model_size)

# Subdividir Audio
Cortar audio en trozos medianos para mejorar el procesamiento.

In [ ]:
!rm clips/*
!rm clips
!mkdir clips
!rm texts/*
!rm texts
!mkdir texts

#import wave
#import contextlib

if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

with contextlib.closing(wave.open(path,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
    duration_total = duration

#!pip install pydub

from pydub import AudioSegment

song = AudioSegment.from_file("audio.wav", format="wav")
#with AudioSegment.from_file("audio.wav", format="wav") as song:
# PyDub handles time in milliseconds
seg = 1000

clips = []
while len(clips) * clips_time_min * 60 < duration:
  len_clips = len(clips)
  clips.append({
      "start": len_clips * clips_time_min * 60 * seg,
      "end": min((len_clips+1) * clips_time_min * 60 * seg, duration * seg),
      "file-clip": direction_clips+"clip-"+str(len_clips)+".wav",
      "file-txt": direction_text+"part-"+str(len_clips)+".txt"
  })


for i, clip in enumerate(clips):

  print("init cut clip " + str(i))
  print(clip)
  start = clip["start"]
  end = clip["end"]
  name_clip = clip["file-clip"]
  clip_sound = song[start:end]
  #with song[start:end] as clip_sound:
  clip_sound.export(name_clip, format="wav")
  print(name_clip)

song = None
clip_sound = None

# Deteccion
1. Se detecta el texto hablado en los audios
2. Se agrupan los textos y audios
3. Se realiza una Clusterización para identificar los posibles hablantes
4. Se ejecuta el cluster en cada segmento de audio para detectar al hablante

In [ ]:
def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

def time(secs):
  return datetime.timedelta(seconds=round(secs))



segments_group = []
segment_embedding_group = []
for y, clip in enumerate(clips):
  print("Clip "+ str(y)+ " - " + str(y+1)+"/"+str(len(clips)) + " workinggg....")
  path = clip["file-clip"]

  result = model.transcribe(path)
  segments = result["segments"]

  with contextlib.closing(wave.open(path,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
  audio = Audio()
  for i, segment in enumerate(segments):
    segment_embedding_group.append(segment_embedding(segment))
  for i in range(len(segments)):
    segments[i]["start"] += clips_time_min*y*60
    segments[i]["end"] += clips_time_min*y*60
  segments_group += segments


embeddings = np.zeros(shape=(len(segments_group), 192))
for i, segment in enumerate(segments_group):
  embeddings[i] = segment_embedding_group[i]

embeddings = np.nan_to_num(embeddings)
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments_group)):
  segments_group[i]["speaker"] = 'SPEAKER' + str(labels[i] + 1)







# Generación de Texto y HTML

# Text Generator

In [ ]:
ft = open(direction_text+"translation.txt", "w", encoding='utf-8')
ft.write('Atencion: Documento transcrito por secciones e IA, Revisar\n')
ft.write('Datos:\n')
ft.write(f'- Version Transcriptior: v{version_tscript}\n')
ft.write('- Nombre archivo original: '+ nombre_archivo +'\n')
ft.write('- Duracion de la grabacion: '+ str(time(duration_total)) +'\n')
ft.write('- Duracion maxima de cada clip: '+ str(time(clips_time_min * 60)) +'\n')
ft.write('- Tamaño del modelo de Whisper: '+ model_size +'\n')
ft.write('- Idioma: '+ language +'\n')
ft.write('- Numero hablantes: '+ str(num_speakers) +'\n')

for (i, segment) in enumerate(segments_group):
  if i == 0 or segments_group[i - 1]["speaker"] != segment["speaker"]:
    #f.write("\n" + segment["speaker"] + ' ClipTime: ' + str(time(segment["start"])))
    ft.write("\n" + segment["speaker"] + '    Time: ' + str(time(segment["start"])) + '\n')
  ft.write(segment["text"][1:] + ' ')

ft.close()

In [ ]:
#!zip -r outs.zip texts clips
#!zip -r out_texts.zip texts
files.download(direction_text+"translation.txt")

# HTML Generator


In [ ]:
ft = open(direction_text+"translation.html", "w", encoding='utf-8')
ft.write('<!DOCTYPE html><html lang="es"><head> <meta charset="UTF-8"> <meta name="viewport" content="width=device-width, initial-scale=1.0"><title>Document</title></head><body>')
ft.write('<h3>Atencion: Documento transcrito por secciones e IA, Revisar</h3>\n')
ft.write('<h3>Datos:</h3>\n')
ft.write('<ul>\n')
ft.write(f'<li>Version Transcriptior: v{version_tscript}</li>\n')
ft.write('<li>Nombre archivo original: '+ nombre_archivo +'</li>\n')
ft.write('<li>Duracion de la grabacion: '+ str(time(duration_total)) +'</li>\n')
ft.write('<li>Duracion maxima de cada clip: '+ str(time(clips_time_min * 60)) +'</li>\n')
ft.write('<li>Tamaño del modelo de Whisper: '+ model_size +'</li>\n')
ft.write('<li>Idioma: '+ language +'</li>\n')
ft.write('<li>Numero hablantes: '+ str(num_speakers) +'</li>\n')
ft.write('</ul><p>')

for (i, segment) in enumerate(segments_group):
  if i == 0 or segments_group[i - 1]["speaker"] != segment["speaker"]:
    ft.write("\n</p><p>" + segment["speaker"] + '    Time: ' + str(time(segment["start"])) + '</p>\n<p>')
  ft.write('<span style="color: orange;">[SP.{}={}-{}]</span> '.format(segment["speaker"][-1], str(time(segment["start"])), str(time(segment["end"])))+segment["text"][1:] + ' ')
ft.write('</p></body></html>')

ft.close()

In [ ]:

files.download(direction_text+"translation.html")